In [ ]:
#hide
#default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [ ]:
#export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

from collections import defaultdict
from pprint import pformat
from inspect import signature,Parameter
import ast,contextlib,copy

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

## NBProcessor -

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')

In [ ]:
#export
def extract_comments(ss):
    "Take leading comments from lines of code in `ss`, remove `#`, and split"
    ss = ss.splitlines()
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or re.match('\s*[^#\s]', o))
    return L((s.strip()[1:]).strip().split() for s in ss[:first_code]).filter()

nbprocess comments start with `#`, followed by whitespace delimited tokens, which `extract_comments` extracts from the start of a cell, up until a blank line or a line containing something other than comments:

In [ ]:
exp  = """#export module
# hide
1+2
#bar"""
test_eq(extract_comments(exp), [['export', 'module'],['hide']])

In [ ]:
#export
@functools.lru_cache(maxsize=None)
def _param_count(f):
    "Number of parameters accepted by function `f`"
    params = list(signature(f).parameters.values())
    # If there's a `*args` then `f` can take as many params as needed
    if first(params, lambda o: o.kind==Parameter.VAR_POSITIONAL): return 99
    return len([o for o in params if o.kind in (Parameter.POSITIONAL_ONLY,Parameter.POSITIONAL_OR_KEYWORD)])

In [ ]:
#export
class NBProcessor:
    "Process cells and nbdev comments in a notebook"
    def __init__(self, path=None, procs=None, preprocs=None, postprocs=None, nb=None, debug=False):
        self.nb = read_nb(path) if nb is None else nb
        self.procs,self.preprocs,self.postprocs = map(L, (procs,preprocs,postprocs))
        self.debug = debug

    def _process_cell(self, cell):
        self.cell = cell
        cell._comments = extract_comments(cell.source)
        for proc in self.procs:
            if callable(proc): proc(cell)
            if cell.cell_type=='code': cell._comments.map(self._process_comment,proc)

    def _process_comment(self, proc, comment):
        cmd,*args = comment
        f = getattr(proc, f'_{cmd}_', None)
        if not f or _param_count(f)-1<len(args): return True
        if self.debug: print(cmd, args, f)
        return f(self, *args)
        
    def process(self):
        "Process all cells with `process_cell`"
        for proc in self.preprocs:
            nb = proc(self.nb)
            if nb: self.nb = nb
        for i in range_of(self.nb.cells): self._process_cell(self.nb.cells[i])
        self.nb.cells = [c for c in self.nb.cells if c.source is not None]
        for proc in self.postprocs:
            nb = proc(self.nb)
            if nb: self.nb = nb

In [ ]:
class _PrintExample:
    def _printme_(self, nbp, to_print): print(to_print)

everything_fn = '../tests/01_everything.ipynb'
proc = NBProcessor(everything_fn, _PrintExample())
proc.process()

testing


## Export -

In [ ]:
#skip
basic_export_nb2('00_read.ipynb', 'read')
basic_export_nb2('01_maker.ipynb', 'maker')
basic_export_nb2('02_process.ipynb', 'process')

g = exec_new('import nbprocess.process')
assert hasattr(g['nbprocess'].process, 'NBProcessor')